# Boilerplate Code

In [1]:
import requests
import json
import os
import time
from requests.exceptions import ConnectionError

# you'll need to have an API key for TMDB
# to run these examples,
# run export TMDB_API_KEY=<YourAPIKey>
tmdb_api_key = '321ea1a12b9a5472be76e6b1ed2db21b'
# Setup tmdb as its own session, caching requests
# (we only want to cache tmdb, not elasticsearch)
# Get your TMDB API key from
#  https://www.themoviedb.org/documentation/api
# then in shell do export TMDB_API_KEY=<Your Key>
tmdb_api = requests.Session()
tmdb_api.params={'api_key': tmdb_api_key}

# Getting a List of Movies

In [2]:
def movieList(maxMovies=10000):
    url = 'https://api.themoviedb.org/3/movie/top_rated'
    movieIds = [];
    numPages = int(maxMovies / 20)
    for page in range(1, numPages + 1): #A
        if page % 10 == 0:
            time.sleep(3)  # Sleep for 3 seconds every tenth request
        try:
            httpResp = tmdb_api.get(url, params={'page': page})  #B
        except Exception as e:
            print(e)            
            print(len(movieIds))
        jsonResponse = json.loads(httpResp.text) #C
        movies = jsonResponse['results']
        for movie in movies: #D
            movieIds.append(movie['id'])
    return movieIds

# Extract Using TMDB

In [3]:
def getCastAndCrew(movieId, movie):
    httpResp = tmdb_api.get("https://api.themoviedb.org/3/movie/%s/credits?language=pt-BR" % movieId) 
    credits = json.loads(httpResp.text) #C
    try:
        crew = credits['crew']
        directors = []
        for crewMember in crew: #D
            if crewMember['job'] == 'Director':
                directors.append(crewMember)
    except KeyError as e:
        print(e)
        print(credits)
    movie['cast'] = credits['cast'] #E
    movie['directors'] = directors 

def extract(movieIds=[], numMovies=10000):
    movieDict = {}
    for idx, movieId in enumerate(movieIds):
        try:
            httpResp = tmdb_api.get("https://api.themoviedb.org/3/movie/%s?language=pt-BR" % movieId)
            movie = json.loads(httpResp.text)
            getCastAndCrew(movieId, movie)               
            movieDict[movieId] = movie
            if idx%100 == 0:
                print('Vou dormir 6 segundos...')
                time.sleep(6)
        except ConnectionError as e:
            print(e)
    return movieDict

In [4]:
movieIds = movieList()

In [5]:
movieDict = extract(movieIds)

Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...
Vou dormir 6 segundos...


In [6]:
import pickle 
pickle.dump(movieDict,open("../Dados/movies.p","wb"))